# Visa Certification Analysis — NCGA, Stan Eury, etc.

The analysis below supports several passages in the [BuzzFeed News article, "The Coyote,"](http://www.buzzfeed.com/kenbensinger/the-coyote) published December 29, 2015. Notes:

- These analyses use visa-certification data published by the [Department of Labor's Office of Foreign Labor Certification (OFLC)](https://www.foreignlaborcert.doleta.gov/performancedata.cfm). The OFLC publishes spreadsheets detailing its decisions about whether to approve ("certify") employers to bring H-2 guest workers to the United States. H-2 visas come in two types: H-2A for agricultural workers and H-2B for non-agricultural unskilled workers. The Department of Labor's data covers __H-2A decisions since FY 2006__ and __H-2B decisions since FY 2000__. The most recent data, for both visa types, includes data  __through FY 2015__, which concluded on Sept. 30, 2015.


- The raw data were collected from two sites — [doleta.gov](http://www.foreignlaborcert.doleta.gov/performancedata.cfm) and [fldatacenter.com](http://www.flcdatacenter.com/) — and were then processed into a standardized format. You can find that raw data, the processing scripts, and additional details [here](https://github.com/buzzfeednews/H-2-certification-data).


- The first section of the analysis loads the data, and identifies visa certifications associated with the North Carolina Growers Association and other Eury businesses.


- The remaining sections use the data to support specific passages.

# Data preparation

## Load visa-certification data

In [1]:
import pandas as pd

In [2]:
visa_decisions = pd.read_csv("../data/H-2-certification-decisions.csv", low_memory=False)\
    .rename(columns=dict(fy="fiscal_year"))

In [3]:
to_calendar_year = lambda x: pd.to_datetime(x).year if pd.notnull(x) else None
visa_decisions["calendar_year"] = visa_decisions["last_event_date"].apply(to_calendar_year)

## Identify certifications associated with NCGA and other companies connected to Stan Eury

This analysis focuses on three Eury-controlled businesses: the North Carolina Growers Association, International Labor Management Corp., and The Labor Company. The "agent name" and "lawfirm name" fields in the Department of Labor data are entered free-form, so we use a variety of patterns — e.g., agent/lawfirm names that include "NORTH CAROLINA GROWER" — to identify the applications.

### North Carolina Growers Association (NCGA)

In [4]:
ncga_pat = r"N\.?C\.? GROWER|NCGA|N C G A|NORTH CAROLINA GROWER|STAN.* EURY.* EXEC"

visa_decisions["is_ncga"] = (
    visa_decisions["agent_name"].str.contains(ncga_pat, na=False) |
    visa_decisions["employer_name"].str.contains(ncga_pat, na=False)
)

# Note: On some multi-employer applications, 
# the agent name is listed only on the master application,
# and not on the sub-applications.
ncga_case_numbers = visa_decisions[visa_decisions["is_ncga"]]["case_no"].unique()

visa_decisions.loc[(
    visa_decisions["case_no"].isin(ncga_case_numbers) &
    visa_decisions["agent_name"].isnull()
), "is_ncga"] = True

### International Labor Management Corp. (ILMC)

In [5]:
ilmc_pat = r"INT.*L.? LABOR MAN|INT.*L.? LABOR MGM?T|I\.?L\.?M\.?C\.?|ILM CORP|SARA.* FAR?REL?L"

In [6]:
visa_decisions["is_ilmc"] = (
    visa_decisions["agent_name"].str.contains(ilmc_pat, na=False) |
    visa_decisions["employer_name"].str.contains(ilmc_pat, na=False) |
    (
        # In FY 2013, the DOL data does not include the business name for agents.
        # But the following agents are known to have worked for ILMC that fiscal year.
        visa_decisions["agent_name"].str.contains(r"THERESA.* WARD|CHRISTI TABAR*|DANIEL FARRELL", na=False) &
        (visa_decisions["fiscal_year"] == 2013)
    )
)

ilmc_case_numbers = visa_decisions[visa_decisions["is_ilmc"]]["case_no"].unique()

visa_decisions.loc[(
    visa_decisions["case_no"].isin(ilmc_case_numbers) &
    visa_decisions["agent_name"].isnull()
), "is_ilmc"] = True

### The Labor Company

In [7]:
labor_company_pat = r"THE LABOR COMP"
visa_decisions["is_the_labor_company"] = (
    visa_decisions["agent_name"].str.contains(labor_company_pat, na=False) |
    visa_decisions["employer_name"].str.contains(labor_company_pat, na=False) |
    (
        visa_decisions["agent_name"].str.contains(r"EVELYN .*WRIGHT|AMANDA .*WRIGHT", na=False) &
        (visa_decisions["fiscal_year"] >= 2007)
    )
)

the_labor_company_case_numbers = visa_decisions[visa_decisions["is_the_labor_company"]]["case_no"].unique()

visa_decisions.loc[(
    visa_decisions["case_no"].isin(the_labor_company_case_numbers) &
    visa_decisions["agent_name"].isnull()
), "is_the_labor_company"] = True

### Other Eury-associated certification applications

In [8]:
eury_pat = r"STAN.*EURY|CRAIG.*EURY"

In [9]:
visa_decisions["is_eury_related"] = (
    visa_decisions["is_ncga"] |
    visa_decisions["is_ilmc"] |
    visa_decisions["is_the_labor_company"] |
    visa_decisions["agent_name"].str.contains(eury_pat, na=False)
)

In [10]:
eury_case_numbers = visa_decisions[visa_decisions["is_eury_related"]]["case_no"].unique()
visa_decisions.loc[(
    visa_decisions["case_no"].isin(eury_case_numbers) &
    visa_decisions["agent_name"].isnull()
), "is_eury_related"] = True

The code below [outputs a spreadsheet](https://github.com/BuzzFeedNews/2015-12-ncga-eury/tree/master/output/eury-related-agents.csv) of all the visa agents listed on applications BuzzFeed News has classified as `is_eury_related`.

In [11]:
visa_decisions[
    visa_decisions["is_eury_related"]
].groupby([ "agent_name" ])["n_certified"].sum()\
    .to_frame()\
    .to_csv("../output/eury-related-agents.csv", encoding="utf-8")

# Analyses

## "[...] over more than a quarter of a century, [Eury] has been responsible for procuring hundreds of thousands of Mexican workers on H-2 visas [...]"

The Office of Foreign Labor Certification has published data on H-2A certification decisions since FY 2006 and on H-2B since FY 2000. But even in the limited time for which data on both visa types are available — FY 2006 - FY 2015 — Eury-controlled businesses have received approval for at least 213,000 H-2 visas:

In [12]:
eury_count_since_fy2006 = visa_decisions[
    (visa_decisions["is_eury_related"] == True) &
    (visa_decisions["is_duplicate"] != True) &
    (visa_decisions["fiscal_year"] >= 2006)
]["n_certified"].sum()
eury_count_since_fy2006

213899

In [13]:
all_count_since_fy2006 = visa_decisions[
    (visa_decisions["is_duplicate"] != True) &
    (visa_decisions["fiscal_year"] >= 2006)
]["n_certified"].sum()

print((
"Eury's businesses have received approximately {0:.1f}% "
"of all H-2 certifications issued since FY 2006."
).format(100 * eury_count_since_fy2006 / all_count_since_fy2006))

Eury's businesses have received approximately 9.2% of all H-2 certifications issued since FY 2006.


## "Last year, Eury’s companies were approved for more than 20,000 visas [...]"

In [14]:
visa_decisions[
    (visa_decisions["is_eury_related"]) &
    (visa_decisions["is_duplicate"] != True) &
    (visa_decisions["calendar_year"] >= 2010)
].groupby(["calendar_year"])["n_certified"].sum()

calendar_year
2010    18394
2011    18085
2012    18991
2013    19773
2014    20216
2015    13817
Name: n_certified, dtype: int64

## "Soon [ILMC] was helping clients gain approval for thousands of visas every year."

In [15]:
pd.DataFrame(visa_decisions[
    (visa_decisions["is_ilmc"]) &
    (visa_decisions["visa_type"] == "H-2B") &
    (visa_decisions["is_duplicate"] != True) &
    (visa_decisions["fiscal_year"] <= 2009)
].groupby([ "fiscal_year" ])["n_certified"].sum())

,n_certified
fiscal_year,
2000,9396
2001,4371
2002,4271
2003,4795
2004,4796
2005,5342
2006,5139
2007,4776
2008,5200


## "[...] the North Carolina Growers Association often requested more than 80% of all agricultural guest worker visas in North Carolina."

Percentage of NC-worksite H-2A visas certified for NCGA and NCGA clients, by fiscal year:

In [16]:
grp = visa_decisions[
    (visa_decisions["visa_type"] == "H-2A") &
    (visa_decisions["worksite_state"] == "NC") &
    (visa_decisions["is_duplicate"] != True) &
    (visa_decisions["fiscal_year"] >= 2007)
].groupby([ "fiscal_year", "is_ncga" ])

In [17]:
nc_certified_by_fy = grp["n_certified"].sum()\
    .unstack()\
    .fillna(0)\
    .rename(columns={ True: "NCGA", False: "non-NCGA" })
nc_certified_by_fy["total"] = nc_certified_by_fy.sum(axis=1)
nc_certified_by_fy["NCGA_pct"] = (100.0 * nc_certified_by_fy["NCGA"] / nc_certified_by_fy["total"]).round(1)
nc_certified_by_fy

is_ncga,non-NCGA,NCGA,total,NCGA_pct
fiscal_year,,,,
2007,1633,14379,16012,89.8
2008,1547,7580,9127,83.1
2009,1486,7242,8728,83.0
2010,1418,8375,9793,85.5
2011,1260,7877,9137,86.2
2012,1841,8657,10498,82.5
2013,2749,9637,12386,77.8
2014,3714,11421,15135,75.5
2015,5942,11754,17696,66.4


Many years of the OFLC data lack information on the number of workers requested. Partial certifications, however, are fairly rare, and the proportions of workers-requested are nearly identical to those of workers-certified for the years for which OFLC publishes both fields:

In [18]:
nc_requested_by_fy = grp["n_requested"].sum()\
    .unstack()\
    .rename(columns={ True: "NCGA", False: "non-NCGA" })
nc_requested_by_fy["total"] = nc_requested_by_fy.sum(axis=1)
nc_requested_by_fy["NCGA_pct"] = (100.0 * nc_requested_by_fy["NCGA"] / nc_requested_by_fy["total"]).round(1)
nc_requested_by_fy.dropna(subset=["total"])

is_ncga,non-NCGA,NCGA,total,NCGA_pct
fiscal_year,,,,
2007,1705,14379,16084,89.4
2011,1305,7891,9196,85.8
2012,1893,8660,10553,82.1
2015,6251,11757,18008,65.3


Note: OFLC data for FY 2006 H-2A data does not include worksite state. Overall pct., Jan. 2007 – Sept. 2015:

In [19]:
grp = visa_decisions[
    (visa_decisions["visa_type"] == "H-2A") &
    (visa_decisions["worksite_state"] == "NC") &
    (visa_decisions["is_duplicate"] != True) &
    (visa_decisions["calendar_year"] >= 2007)
].groupby([ "is_ncga" ])

nc_certified_since_fy2007 = pd.DataFrame(grp["n_certified"].sum()\
    .fillna(0)).T\
    .rename(columns={ True: "NCGA", False: "non-NCGA" })
nc_certified_since_fy2007["total"] = nc_certified_since_fy2007.sum(axis=1)
nc_certified_since_fy2007["NCGA_pct"] = (100.0 * nc_certified_since_fy2007["NCGA"] / nc_certified_since_fy2007["total"]).round(1)
nc_certified_since_fy2007

is_ncga,non-NCGA,NCGA,total,NCGA_pct
n_certified,21584,86888,108472,80.1


## "For years, employers in North Carolina obtained approvals for more such guest worker visas than employers in any other state."

Number of H-2A visas per worksite-state and fiscal year:

In [20]:
fiscal_year_by_state = visa_decisions[
    (visa_decisions["is_duplicate"] != True) &
    (visa_decisions["visa_type"] == "H-2A")
].groupby([ "worksite_state", "fiscal_year" ])["n_certified"].sum()\
    .unstack().fillna(0)

In [21]:
fiscal_year_by_state.tail()

fiscal_year,2007,2008,2009,2010,2011,2012,2013,2014,2015
worksite_state,,,,,,,,,
VT,450,501,452,436,449,418,465,496,507
WA,1994,2573,1984,3013,3194,4443,6251,9064,11844
WI,170,332,391,141,126,171,176,492,448
WV,39,37,48,42,38,134,34,37,72
WY,494,396,358,385,279,319,301,309,326


North Carolina's rank, by fiscal year:

In [22]:
pd.DataFrame(fiscal_year_by_state.rank(ascending=False).ix["NC"])

,NC
fiscal_year,
2007,1
2008,1
2009,1
2010,1
2011,1
2012,1
2013,1
2014,1
2015,2


## "In the 12-month period starting July 2014, the growers association got permission to import more than 12,000 guest workers for jobs in North Carolina."

In [23]:
visa_decisions[
    visa_decisions["is_ncga"] &
    (visa_decisions["is_duplicate"] != True) &
    (visa_decisions["last_event_date"] >= "2014-07-01") &
    (visa_decisions["last_event_date"] < "2015-07-01")
]["n_certified"].sum()

12034

## "Even with more competition, in 2011, [Eury] was still responsible for securing one in eight of all H-2 guest worker visa certifications."

In [24]:
eury_by_year = visa_decisions[
    (visa_decisions["is_duplicate"] != True) &
    (visa_decisions["calendar_year"] >= 2009)
].groupby([ "calendar_year", "is_eury_related" ])["n_certified"].sum()\
    .unstack()\
    .fillna(0)\
    .rename(columns={ True: "Eury", False: "non-Eury" })
eury_by_year["total"] = eury_by_year.sum(axis=1)
eury_by_year["eury_pct"] = (100.0 * eury_by_year["Eury"] / eury_by_year["total"]).round(1)
eury_by_year

is_eury_related,non-Eury,Eury,total,eury_pct
calendar_year,,,,
2009,193390,23955,217345,11.0
2010,144908,18394,163302,11.3
2011,126517,18085,144602,12.5
2012,154093,18991,173084,11.0
2013,154983,19773,174756,11.3
2014,205852,20216,226068,8.9
2015,177008,13817,190825,7.2


## "In a typical year, [Eury's] companies meet the labor needs of clients across more than 20 states."

In [25]:
grp = visa_decisions[
    (visa_decisions["is_eury_related"] == True) &
    (visa_decisions["n_certified"] > 0) &
    (visa_decisions["worksite_state"].notnull())
].groupby([ "fiscal_year" ])
pd.DataFrame({
    "states": grp.apply(lambda x: " • ".join(sorted(x["worksite_state"].astype(str).unique()))),
    "n_states": grp["worksite_state"].nunique(),
})

,n_states,states
fiscal_year,,
2007,36,AL • AR • CA • CO • CT • FL • GA • IA • IL • I...
2008,32,AL • AR • CA • CO • CT • FL • GA • IA • IL • I...
2009,32,AL • AR • CA • CO • CT • FL • GA • IA • IL • I...
2010,33,AL • AR • CA • CO • DE • FL • GA • IA • IL • I...
2011,31,AL • AR • CA • CO • FL • GA • IA • IL • IN • K...
2012,31,AL • AR • CO • FL • GA • IA • IL • IN • KY • L...
2013,28,AL • AR • CO • FL • GA • IA • IL • IN • KY • L...
2014,30,AL • AR • CA • CO • FL • GA • IA • IL • IN • K...
2015,20,AL • AR • CO • CT • FL • GA • IL • LA • MD • M...


## "The North Carolina Growers Association has continued to get visas approved for Walker Farms, more than 90 since Ordaz died, including 13 this year."

In [26]:
walker_since_fy2006 = visa_decisions[
    visa_decisions["is_ncga"] &
    (visa_decisions["fiscal_year"] >= 2006) &
    (visa_decisions["employer_state"] == "NC") &
    visa_decisions["employer_name"].str.contains(r"DANNY.* WALKER|WALKER FARM", na=False)
].set_index("case_no")

walker_since_fy2006[[
    "employer_name", "employer_state",
    "visa_type", "fiscal_year", "last_event_date", 
    "certification_begin_date", "case_status", "n_certified"
]]

,employer_name,employer_state,visa_type,fiscal_year,last_event_date,certification_begin_date,case_status,n_certified
case_no,,,,,,,,
A-06060-00924,DANNY WALKER,NC,H-2A,2006,NaN,2006-04-15,CERTIFIED - FULL,3
A-06137-01534,DANNY WALKER,NC,H-2A,2006,NaN,NaN,INCOMPLETE,0
A-06138-01537,DANNY WALKER,NC,H-2A,2006,NaN,2006-07-01,CERTIFIED - FULL,3
A-07054-03309,DANNY WALKER,NC,H-2A,2007,2007-03-15,2007-04-12,CERTIFIED - FULL,4
A-07110-03899,DANNY WALKER,NC,H-2A,2007,2007-05-09,2007-06-05,CERTIFIED - FULL,4
A-08053-06049,DANNY WALKER,NC,H-2A,2008,2008-03-11,2008-04-10,CERTIFIED - FULL,4
A-08123-06789,DANNY WALKER,NC,H-2A,2008,2008-05-21,2008-06-19,CERTIFIED - FULL,4
C-09057-18303,DANNY WALKER,NC,H-2A,2009,2009-03-17,2009-04-16,CERTIFIED - FULL,8
C-10050-23238,DANNY WALKER,NC,H-2A,2010,2010-03-09,2010-04-08,CERTIFIED - FULL,5


In [27]:
walker_since_fy2006["n_certified"].sum()

91

In [28]:
walker_since_fy2006[
    walker_since_fy2006["calendar_year"] == 2015
]["n_certified"].sum()

13

##  "International Labor Management was a major player, helping clients get approval for as many as 5,000 visas a year."

In [29]:
pd.DataFrame(visa_decisions[
    (visa_decisions["is_ilmc"]) &
    (visa_decisions["visa_type"] == "H-2B") &
    (visa_decisions["is_duplicate"] != True) &
    (visa_decisions["fiscal_year"] <= 2009)
].groupby([ "fiscal_year" ])["n_certified"].sum())

,n_certified
fiscal_year,
2000,9396
2001,4371
2002,4271
2003,4795
2004,4796
2005,5342
2006,5139
2007,4776
2008,5200


## "But it took the Labor Department 10 days [after ILMC pleaded guilty on July 22nd] to follow through — and in that time the agency approved at least 12 of the company’s visa applications for a total of more than 400 workers."

In [30]:
ilmc_post_20140722 = visa_decisions[
    visa_decisions["is_ilmc"] &
    (visa_decisions["last_event_date"] > "2014-07-22")
].sort_values("last_event_date").set_index("case_no")[[ 
    "visa_type", "last_event_date", "case_status", "n_certified", 
    "employer_name", "agent_name"
]]
ilmc_post_20140722

,visa_type,last_event_date,case_status,n_certified,employer_name,agent_name
case_no,,,,,,
H-300-14184-484539,H-2A,2014-07-23,DETERMINATION ISSUED - CERTIFICATION,56,LAMONT FRUIT FARM INC #3,INTERNATIONAL LABOR MANAGEMENT CORP:THERESA WARD
H-300-14184-758697,H-2A,2014-07-23,DETERMINATION ISSUED - CERTIFICATION,58,MATTHEWS RIDGEVIEW FARM,INTERNATIONAL LABOR MANAGEMENT CORP:CHRISTI TA...
H-300-14192-264473,H-2A,2014-07-24,DETERMINATION ISSUED - CERTIFICATION,5,HARTLAND INC,INTERNATIONAL LABOR MANAGEMENT CORP:TERESA WARD
H-300-14192-685801,H-2A,2014-07-24,DETERMINATION ISSUED - CERTIFICATION,18,THOMAS A & ROBERT E CONROW,INTERNATIONAL LABOR MANAGEMENT CORPORATION:THE...
H-300-14184-120609,H-2A,2014-07-25,DETERMINATION ISSUED - CERTIFICATION,39,KALIR ENTERPRISES INC,INTERNATIONAL LABOR MANAGEMENT CORP:CHRISTI TA...
H-300-14192-166510,H-2A,2014-07-25,DETERMINATION ISSUED - CERTIFICATION,4,L. WALTHERS & SONS INC #12,INTERNATIONAL LABOR MANAGEMENT CORPORATION:THE...
H-300-14192-356336,H-2A,2014-07-25,DETERMINATION ISSUED - CERTIFICATION,16,JAMES KIRBY,INTERNATIONAL LABOR MANAGEMENT CORPORATION:THE...
H-300-14192-907762,H-2A,2014-07-25,DETERMINATION ISSUED - CERTIFICATION,50,LEWIS M BAILEY IV FARMS INC,INTERNATIONAL LABOR MANAGEMENT CORPORATION:THE...
H-300-14184-384344,H-2A,2014-07-28,DETERMINATION ISSUED - CERTIFICATION,33,MATTHEWS SWEET POTATO FARM,INTERNATIONAL LABOR MANGEMENT CORPORATION:CHRI...


In [31]:
len(ilmc_post_20140722)

12

In [32]:
ilmc_post_20140722["n_certified"].sum()

411

## "And [Wicker] kept submitting the association’s visa applications for more than four months — 20 applications in all, for a total of 3,887 workers."

Wicker signed these 21 applications — one of which was withdrawn — between April 10, 2015 and the end of FY 2015:

In [33]:
wicker_case_numbers_since_20150410 = [
    "H-300-15105-204510",
    "H-300-15105-401959",
    "H-300-15113-780489",
    "H-300-15121-107708",
    "H-300-15121-272547",
    "H-300-15126-127520",
    "H-300-15133-717897",
    "H-300-15133-981598",
    "H-300-15146-218107",
    "H-300-15146-896009",
    "H-300-15149-227646",
    "H-300-15154-234368",
    "H-300-15154-808765",
    "H-300-15162-892086",
    "H-300-15168-424111",
    "H-300-15181-041168",
    "H-300-15191-311857",
    "H-300-15197-043964",
    "H-300-15233-161244",
    "H-300-15239-744626",
    "H-300-15239-792339",    
]

In [34]:
len(wicker_case_numbers_since_20150410)

21

The total number of worker positions certified for these applications:

In [35]:
visa_decisions[
    (visa_decisions["is_duplicate"] != True)
].set_index("case_no").ix[wicker_case_numbers_since_20150410]["is_certified"].sum()

20

In [36]:
visa_decisions[
    (visa_decisions["is_duplicate"] != True)
].set_index("case_no").ix[wicker_case_numbers_since_20150410]["n_certified"].sum()

3887

## "Since [August 3, 2014, National Agricultural Consultants] has won approval for 5,100 H-2 visas, federal data show [...]"

Number of certifications per listed agent company/name since July 1, 2014:

In [37]:
nat_ag_pat = r"\bNATIONAL AG|\bNAT[^ ]*L AG"
nat_ag = visa_decisions[
    visa_decisions["agent_name"].str.contains(nat_ag_pat, na=False) &
    (visa_decisions["last_event_date"] >= "2014-08-03") &
    (visa_decisions["is_duplicate"] != True)
]
nat_ag.groupby(["agent_name"])["n_certified"].sum()

agent_name
NATIONAL AGRICULTURAL CONSULTANTS LLC:THERESA WARD     5056
NATIONAL AGRICULTURAL CONSULTANTS LLC:WARREN WICKER      52
NATIONAL AGRICULTURAL CONSULTANTS:THERESA WARD           15
NATIONAL AGRICULTURE CONSULTANTS LLC:THERESA WARD         0
NATIONAL AGRICULURAL CONSULTANTS LLC:THERESA WARD        12
Name: n_certified, dtype: int64

In [38]:
nat_ag["n_certified"].sum()

5135

## "[...] its 10 largest clients had all previously gotten visas through International Labor Management."

In [39]:
import namestand
def normalize_name(x):
    return namestand.downscore(x).replace("_", "")[:10]

NAC's largest clients:

In [40]:
grp = nat_ag.groupby(nat_ag["employer_name"].apply(normalize_name))
pd.DataFrame({
    "n_certified": grp["n_certified"].sum(),
    "employer_names": grp["employer_name"].apply(lambda x: " • ".join(sorted(x.unique()))),
    "employer_zip_codes": grp["employer_postal_code"].apply(lambda x: " • ".join(sorted(x.unique())))
}).sort_values("n_certified", ascending=False).head(12)

,employer_names,employer_zip_codes,n_certified
employer_name,,,
tbelldetas,T. BELL DETASSELING LLC,52739,473
walterpraw,WALTER P. RAWL & SONS INC,29123,324
freybrothe,FREY BROTHERS,62851,224
jecooleyfa,JE COOLEY FARMS INC,29323,215
dutchmantr,DUTCHMAN TREE FARMS LLC,49663,205
joneschurc,JONES & CHURCH FARMS,37692,175
willowbend,WILLOWBEND NURSERIES LLC,44081,174
tpharvesti,T & P HARVESTING LLC • T & P HARVESTING LLC,38878,163
csmcleodfa,C.S MCLEOD FARMS INC,29101,160


Scott Strawberries and Matthews Ridgeview Farms are tied for 10th-largest client through the end of FY 2015. Both have received visa certifications through ILMC, however. Below are examples of visas previously obtained by these clients through ILMC.

In [41]:
nac_prev_ilmc_case_nos = [
    "H-300-14107-535733", "H-300-14156-392963", "H-300-14184-707008", "H-300-14037-080405",
    "H-300-14042-816041", "H-300-14134-615702", "H-300-14007-844145", "H-300-14071-513596",
    "H-300-14037-956780", "H-300-14134-859520", "H-300-14184-758697",
]
visa_decisions[
    visa_decisions["case_no"].isin(nac_prev_ilmc_case_nos)
][[
    "case_no", "visa_type", "last_event_date", "n_certified",
    "employer_name", "employer_postal_code", "agent_name"
]].set_index("case_no").ix[nac_prev_ilmc_case_nos]

,visa_type,last_event_date,n_certified,employer_name,employer_postal_code,agent_name
case_no,,,,,,
H-300-14107-535733,H-2A,2014-06-06,250,T. BELL DETASSELING LLC,52755,INTERNATIONAL LABOR MANAGEMENT CORPORATION:THE...
H-300-14156-392963,H-2A,2014-06-24,40,WALTER P. RAWL & SONS INC,29123,INTERNATIONAL LABOR MANAGEMENT CORPORATION:THE...
H-300-14184-707008,H-2A,2014-07-30,93,FREY BROTHERS,62851,INTERNATIONAL LABOR MANAGEMENT CORPORATION:CHR...
H-300-14037-080405,H-2A,2014-03-11,115,JE COOLEY FARMS INC #2,29323,INTERNATIONAL LABOR MANAGEMENT CORPORATION:THE...
H-300-14042-816041,H-2A,2014-03-25,47,DUTCHMAN TREE FARMS LLC #1,49663,INTERNATIONAL LABOR MANAGEMENT CORP:THERESA WARD
H-300-14134-615702,H-2A,2014-06-06,90,JONES & CHURCH FARMS INC.,37692,INTERNATIONAL LABOR MANAGEMENT CORPORATION:THE...
H-300-14007-844145,H-2A,2014-01-30,22,WILLOWBEND NURSERIES LLC #2,44146,INTERNATIONAL LABOR MANAGEMENT CORPORATION:THE...
H-300-14071-513596,H-2A,2014-04-04,26,"T & P HARVESTING, LLC",38851,INTERNATIONAL LABOR MANAGEMENT CORPORATION:CHR...
H-300-14037-956780,H-2A,2014-03-13,80,C.S.MCLEOD FARMS INC #2,29101,INTERNATIOAL LABOR MANAGEMENT CORPORATION:THER...


## "Since Eury was first indicted [on Jan. 31, 2014], the [NCGA] has won approval for at least 22,000 visas."

In [42]:
visa_decisions[
    visa_decisions["is_ncga"] &
    (visa_decisions["last_event_date"] > "2014-01-31") &
    (visa_decisions["is_duplicate"] != True)
]["n_certified"].sum()

22957

---

---

---